<img width="150" alt="Logo_ER10" src="https://user-images.githubusercontent.com/3244249/151994514-b584b984-a148-4ade-80ee-0f88b0aefa45.png">

## Keras to ONNX conversion
This notebook shows how to convert your trained Keras model to ONNX, the generic format supported by DIANNA. <br>

The conversion is complete with the tf2onnx Python package, which supports both the SavedModel format and the older HDF5 (.h5 or .keras) format. It can convert multi-backend keras as well as tf.keras models.

In [1]:
import os
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
import onnx
import onnxruntime as ort
# In addition to these imports, this notebook
# depends on tf2onnx. It is used from the command line.

2023-12-12 14:12:39.037240: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-12 14:12:39.100024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 14:12:39.100061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 14:12:39.102844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 14:12:39.115682: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-12 14:12:39.116951: I tensorflow/core/platform/cpu_feature_guard.cc:1

Download and initialize built-in model.

In [2]:
model = keras.applications.mobilenet.MobileNet(weights='imagenet')

Evaluate model on some random input.

In [3]:
input_shape = [1] + model.inputs[0].shape[1:]  # input shape without a 1 for batch size, instead of None
input_data = np.random.normal(size=input_shape).astype(np.float32)
pred = model.predict(input_data)

1/1 [==============================] - 1s 983ms/step


Save keras model to SavedModel format.

In [4]:
savedmodel_dir = 'mysavedmodel'
tf.saved_model.save(model, savedmodel_dir)

INFO:tensorflow:Assets written to: mysavedmodel/assets


INFO:tensorflow:Assets written to: mysavedmodel/assets


Convert to ONNX.

In [5]:
onnx_savedmodel = 'mysavedmodel.onnx'
!python -m tf2onnx.convert --saved-model {savedmodel_dir} --output {onnx_savedmodel} --signature_def serving_default --tag serve

2023-12-12 14:13:15.345213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 14:13:15.345304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 14:13:15.346390: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 14:13:16.833234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2

Evaluate ONNX models and compare to keras model output.

In [6]:
# verify the ONNX model is valid
onnx_model = onnx.load(Path(onnx_savedmodel))
onnx.checker.check_model(onnx_model)

# get ONNX predictions
sess = ort.InferenceSession(onnx_savedmodel)
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

onnx_input = {input_name: input_data}
pred_onnx = sess.run([output_name], onnx_input)[0]

print(np.allclose(pred_onnx, pred, atol=1e-5))

True
